In [1]:
!pip install datasets

In [5]:
from datasets import load_dataset

In [6]:
data = load_dataset('majorSeaweed/financeQA_100K')


In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 90566
    })
    validation: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 5031
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 5032
    })
})

In [8]:
data_tr = load_dataset('majorSeaweed/financeQA_100K', split = 'train')
data_val = load_dataset('majorSeaweed/financeQA_100K', split = 'validation')
data_test = load_dataset('majorSeaweed/financeQA_100K', split = 'test')


In [9]:
print(data_tr)
print(data_val)
print(data_test)

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 90566
})
Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 5031
})
Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 5032
})


In [14]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import numpy as np
import pandas as pd
import re

In [12]:
# Take small subsets for fast trainingtrain_data = data['train'].select(range(5000))  # Good for substantial training
train_data = data['train'].select(range(5000))  # Good for substantial training
val_data = data['validation'].select(range(1000))  # Better validation
test_data = data['test'].select(range(1000))  # Proper testing

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")

Training samples: 5000
Validation samples: 1000
Test samples: 1000


In [16]:
# STEP 2: Initialize tokenizer FIRST
from transformers import GPT2Tokenizer

print("Loading tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [20]:
def clean_text(text):
    """Clean text but KEEP stop words for natural conversation"""
    if not isinstance(text, str):
        return ""
    
    # Remove markdown patterns
    text = re.sub(r'#+\s*Document Type[:]?', '', text)
    text = re.sub(r'\*\*.*?\*\*', '', text)
    text = re.sub(r'###\s*', '', text)
    text = re.sub(r'- \*\*', '', text)
    
    # Clean extra whitespace but keep natural language
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def create_conversation(example):
    """Format for conversational training"""
    question = clean_text(example['question'])
    answer = clean_text(example['answer'])
    
    # Simple conversational format
    formatted_text = f"User: {question} Assistant: {answer}{tokenizer.eos_token}"
    return {'text': formatted_text}

print("Cleaning and formatting data...")
train_data_clean = train_data.map(create_conversation)
val_data_clean = val_data.map(create_conversation)
# Show some examples
print("\nSample formatted conversations:")
for i in range(2):
    print(f"Example {i+1}: {train_data_clean[i]['text'][:100]}...")


Cleaning and formatting data...

Sample formatted conversations:
Example 1: User: What is the total estimated project cost mentioned in the document? Assistant: The grand total...
Example 2: User: Where should the payment be remitted to? Assistant: The payment should be remitted to Wolf Kni...


In [21]:
# STEP 4: Tokenization (now tokenizer is defined)
def tokenize_data(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=256,
        return_tensors="tf"
    )

print("Tokenizing data...")
tokenized_train = train_data_clean.map(tokenize_data, batched=True, remove_columns=train_data_clean.column_names)
tokenized_val = val_data_clean.map(tokenize_data, batched=True, remove_columns=val_data_clean.column_names)


Tokenizing data...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
2025-10-08 21:53:22.129163: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]